In [17]:

import torch
import calc_delta_p
import calc_delta_p_v3
# Example Values: ###  #TODO: delete!!!
T = 20
N = 1
H_matrix = torch.randn((T, 3, 3)).cuda()
H_matrix[:, 2, 2] = 1
gx_chosen_values = torch.randn((T, N)).cuda()
gy_chosen_values = torch.randn((T, N)).cuda()
Jx_chosen_values = torch.randn((T, N)).cuda()
Jy_chosen_values = torch.randn((T, N)).cuda()
current_level_reference_tensor_zero_mean = (torch.randn(
    (1, N)).cuda()).repeat(T, 1)  # this should be (1 ,N)
current_level_input_tensor_warped = torch.randn((T, N)).cuda()


In [2]:

def images_and_gradients_to_delta_p_yuri(H_matrix,
                                         current_level_reference_tensor_zero_mean,
                                         current_level_input_tensor_warped,
                                         Jx_chosen_values, Jy_chosen_values,
                                         gx_chosen_values, gy_chosen_values):
    ### [H_matrix] = [T,3,3]
    ### [Jx_chosen_values] = [T,N]
    ### [Jy_chosen_values] = [T,N]
    ### [gx_chosen_values] = [T,N]
    ### [gy_chosen_values] = [T,N]
    ### [current_level_reference_tensor_zero_mean] = [T,N]
    ### [current_level_input_tensor_warped] = [T,N]

    ### Correct dimensions for pytorch arithmatic: ###
    Jx_chosen_values = Jx_chosen_values.unsqueeze(
        1).unsqueeze(1)  # -> [T,N,1,1]
    Jy_chosen_values = Jy_chosen_values.unsqueeze(
        1).unsqueeze(1)  # -> [T,N,1,1]
    H_matrix_corrected = H_matrix.unsqueeze(-1)  # -> [T,3,3,1]
    ### Calculate den once: ###

    den = (H_matrix_corrected[:, 2:3, 0:1] * Jx_chosen_values +
           H_matrix_corrected[:, 2:3, 1:2] * Jy_chosen_values +
           H_matrix_corrected[:, 2:3, 2:3])
    denom_inverse = 1 / den

    ### H Transform xy_prime values: ###
    xy_prime_reshaped_X = (H_matrix_corrected[:, 0:1, 0:1] * Jx_chosen_values +
                           H_matrix_corrected[:, 0:1, 1:2] * Jy_chosen_values +
                           H_matrix_corrected[:, 0:1, 2:3]) * denom_inverse
    xy_prime_reshaped_Y = (H_matrix_corrected[:, 1:2, 0:1] * Jx_chosen_values +
                           H_matrix_corrected[:, 1:2, 1:2] * Jy_chosen_values +
                           H_matrix_corrected[:, 1:2, 2:3]) * denom_inverse

    ### Correct Jx,Jy values: ###
    Jx_chosen_values = Jx_chosen_values * denom_inverse  # element-wise
    Jy_chosen_values = Jy_chosen_values * denom_inverse  # element-wise

    #### Get final Jxx,Jxy,Jyy,Jyx values: ####
    Jxx_prime = Jx_chosen_values * xy_prime_reshaped_X  # element-wise.
    Jyx_prime = Jy_chosen_values * xy_prime_reshaped_X
    Jxy_prime = Jx_chosen_values * xy_prime_reshaped_Y  # element-wise
    Jyy_prime = Jy_chosen_values * xy_prime_reshaped_Y

    # ### Get final jacobian of the H_matrix with respect to the different parameters: ###
    # J_list = [Jx_chosen_values, Jy_chosen_values, Jxx_prime, Jxy_prime, Jyx_prime, Jyy_prime]

    ### Yuri calculations: ###
    current_level_reference_tensor_zero_mean = current_level_reference_tensor_zero_mean.unsqueeze(
        1).unsqueeze(1)  # ->[T,1,1,N]
    current_level_input_tensor_warped = current_level_input_tensor_warped.unsqueeze(
        1).unsqueeze(1)  # ->[T,1,1,N]
    gx_chosen_values = gx_chosen_values.unsqueeze(
        1).unsqueeze(1)  # ->[T,1,1,N]
    gy_chosen_values = gy_chosen_values.unsqueeze(
        1).unsqueeze(1)  # ->[T,1,1,N]

    G, Gt, Gw, C ,delta_p = calc_delta_p.ecc_calc_delta_p(gx_chosen_values, gy_chosen_values,
                                            Jx_chosen_values, Jy_chosen_values,
                                            Jxx_prime, Jxy_prime, Jyx_prime, Jyy_prime,
                                            current_level_reference_tensor_zero_mean,
                                            current_level_input_tensor_warped)
    return G, Gt, Gw, C ,delta_p



In [18]:
G1, Gt1, Gw1, C1 ,delta_p1 = images_and_gradients_to_delta_p_yuri(H_matrix,
                                                current_level_reference_tensor_zero_mean,
                                                current_level_input_tensor_warped,
                                                Jx_chosen_values, Jy_chosen_values,
                                                gx_chosen_values, gy_chosen_values)

In [19]:
G2, Gt2, Gw2, C2 ,delta_p2 = calc_delta_p_v3.ecc_calc_delta_p(H_matrix,
                                            current_level_reference_tensor_zero_mean,
                                            current_level_input_tensor_warped,
                                            Jx_chosen_values, Jy_chosen_values,
                                            gx_chosen_values, gy_chosen_values)

In [20]:
# print ( (0.5*(delta_p2-delta_p1)/ (delta_p2.abs()+delta_p1.abs())).abs().argmax(dim=0))
print ( (0.5*(delta_p2-delta_p1)/ (delta_p2.abs()+delta_p1.abs())).abs().max(dim=0))

torch.return_types.max(
values=tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [   nan],
        [   nan],
        [0.5000]], device='cuda:0'),
indices=tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1]], device='cuda:0'))


In [26]:
print ( (0.5*(delta_p2-delta_p1)/ (delta_p2.abs()+delta_p1.abs())).abs())

tensor([[[0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [   nan],
         [   nan],
         [0.1667]],

        [[0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.1667],
         [0.5000],
         [0.0786],
         [0.5000]],

        [[0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000]],

        [[0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.1957],
         [0.5000],
         [0.5000],
         [0.1667]],

        [[0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000]],

        [[0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000]],

        [[0.3333],
         [0.5000],
         [0.0499],
         [0.5000],


In [29]:
dC=C1-C2
print(dC.max())


tensor(1.7166e-05, device='cuda:0')


In [29]:
print((G1[15]-G2[15]).argmax(dim=0))

tensor([11684, 16942, 12902, 11684, 16942, 12902,     0,     0],
       device='cuda:0')


In [33]:
print((G1[15,11684]-G2[15,11684]))
# print(G1[15,11684])
# print(G2[15,11684])

tensor([ 4.1687e-02, -3.4576e-02,  1.7700e+02,  2.8290e-02, -2.3468e-02,
         1.2000e+02,  0.0000e+00,  0.0000e+00], device='cuda:0')


In [25]:
print(G1)
print(G2)



# print(G1-G2)


tensor([[[-1.6413e-01,  3.9441e-02, -2.1292e-01, -5.4580e-01,  1.3116e-01,
          -7.0804e-01, -1.6917e+00,  4.0654e-01]],

        [[-4.1696e-01, -8.9444e-01,  1.0143e+00, -5.7803e-01, -1.2400e+00,
           1.4061e+00, -5.8199e-01, -1.2485e+00]],

        [[ 2.9093e-02, -1.7680e-01,  2.2613e-01,  5.2972e-02, -3.2191e-01,
           4.1172e-01,  2.0667e-01, -1.2560e+00]],

        [[-9.4748e-02,  1.8786e-02, -9.3562e-02, -1.1380e+00,  2.2563e-01,
          -1.1237e+00,  1.1616e+00, -2.3031e-01]],

        [[-2.7769e-01,  2.2094e-01,  1.6930e-02, -3.5279e-01,  2.8069e-01,
           2.1509e-02,  1.0175e+00, -8.0960e-01]],

        [[ 1.5558e-01,  1.5574e+00, -2.8610e+01,  5.1485e-01,  5.1537e+00,
          -9.4676e+01,  7.3847e-02,  7.3921e-01]],

        [[ 4.2334e-01,  2.4816e-01,  1.0301e-01,  1.7569e-01,  1.0299e-01,
           4.2748e-02,  1.3477e+00,  7.8999e-01]],

        [[ 2.1419e-01,  9.5646e-01, -3.1301e-01, -2.5204e-01, -1.1255e+00,
           3.6832e-01, -2.6113e-01, 

In [50]:
print((G1[15]-G2[15]))
print(())

tensor(80907.1250, device='cuda:0')
()


In [30]:
i_C1 = torch.linalg.inv(C1)
i_C2 = torch.linalg.inv(C2)


In [34]:
(i_C2-i_C1).max()

tensor(1.5616e+10, device='cuda:0')